In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df = pd.read_csv('/kaggle/input/depression-train/train_data.csv')

In [5]:
train_df.head(15)

,PID,Text data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate
5,train_pid_6,I will probably end it when my mum isn't aroun...,moderate
6,train_pid_7,Fuck 2019 : Left abusive relationship. Moved i...,moderate
7,train_pid_8,I am at a new year's eve party and I want to c...,moderate
8,train_pid_9,Death of my father : My father died in the beg...,moderate
9,train_pid_10,Empty and stuck in a loop every day : In any o...,moderate


In [10]:
train_df['Label'].value_counts()

moderate          3678
not depression    2755
severe             768
Name: Label, dtype: int64

In [11]:
!pip install transformers
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [12]:
classifier("a good movie")

[{'label': 'POSITIVE', 'score': 0.999862790107727}]

In [14]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [15]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name)

# Tokenize the data
max_length = 128

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [16]:
def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors="tf"
    )


In [17]:
x = train_df["Text data"].tolist()
tokens = [tokenize_text(text) for text in x]

In [22]:
tokens[0]

{'input_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[  101,  3403,  2005,  2026,  2568,  2000,  2031,  1037, 12554,
         2320,  1996,  1523,  2047,  2095,  1524,  3110,  3475,  1521,
         1056,  2045,  4902,  1024,  1045,  2123,  1521,  1056,  2113,
         2055,  3087,  2842,  1010,  2021,  1045,  1521,  1049,  1037,
         2210,  2978,  5191,  2008,  1045,  1521,  2222,  2175,  2067,
         2000,  2108, 14777,  1999,  1037,  2261,  2420,  2051,  2030,
         2242,  1012,  2197,  2095,  1010,  1045,  2699,  2025,  2000,
         2031,  2151, 12554,  2015,  2005,  1996,  2707,  1997, 10476,
         1012,  1037,  8210,  2184,  2420,  2101,  1010,  1045,  3631,
         2091,  6933,  1012,  1045,  2347,  1521,  1056,  1996,  2168,
         2005,  2008,  2972,  2095,  1012,  2039,  2127,  2285,  1010,
         2073,  1045,  2001,  7929,  2008,  3204,  1012,  2085,  1045,
         2074,  3524,  1012,  1012,  1012,  2009,  1521,  1055,  1037,
         6881, 

In [20]:
label_to_id = {"moderate": 0, "not depression": 1, "severe": 2}
y = train_df["Label"].apply(lambda x: label_to_id[x]).tolist()

In [23]:
y[0]

0

In [24]:
input_ids = np.vstack([t["input_ids"] for t in tokens])
attention_mask = np.vstack([t["attention_mask"] for t in tokens])
labels = np.array(y)

In [25]:
outputs = model.predict([input_ids, attention_mask])[0]
predicted_labels = np.argmax(outputs, axis=1)

226/226 [==============================] - 788s 3s/step


In [26]:
accuracy = (predicted_labels == labels).mean()
print(accuracy)

0.526732398278017
